### 00 OpenAI API KEY 환경변수 등록

In [1]:
import os, json

with open('conf.json', 'r') as f:
        json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['openai_config']["API_KEY"]

### 01 설정

In [2]:
class CONFIG:
    etf_tkr="AIQ"

In [3]:
from data.get_data import get_pdf_df

pdf_df = get_pdf_df(etf_tkr=CONFIG.etf_tkr)

tkr_list = pdf_df.child_stk_tkr.to_list()

### 02 영향도 상위 5개 기업 추출

#### 02-1 2분기 가격데이터 로드

In [4]:
from data.get_data import get_prices_df
from data.fetch_data import fetch_data_from_db

prices_df = get_prices_df(tickers=tkr_list, start_date="2023-03-01")

#### 02-2 유동주식수 로드

In [5]:
import pandas as pd

sql = """
    SELECT * FROM os_stk_info;
"""

stk_info_df = pd.DataFrame(fetch_data_from_db(query=sql)).dropna()

stk_info_df = stk_info_df[stk_info_df["stk_tkr"].isin(prices_df.columns.str.upper().to_list())].reset_index(drop=True)

stk_info_df.index = stk_info_df['stk_tkr'].str.lower()

stk_info_df = stk_info_df[['float_shares']]

#### 02-3 영향도 상위 5개 기업 산출

In [6]:
시작_영향도 = stk_info_df['float_shares'] * prices_df.iloc[0]

끝_영향도 = stk_info_df['float_shares'] * prices_df.iloc[-1]

영향도 = abs(끝_영향도 - 시작_영향도)

영향도_top5_종목들_리스트 = 영향도.sort_values(ascending=False).head().index.str.upper().to_list()

영향도_top5_종목들_리스트

['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN']

### 03 뉴스데이터 로드

In [7]:
import yfinance as yf
from tqdm.auto import tqdm

stk_infos = yf.Tickers(영향도_top5_종목들_리스트)

display(tkr_list[0])

text = {
    f"{CONFIG.etf_tkr} news": []
}

for tkr in tqdm(영향도_top5_종목들_리스트):
    
    tmp = {}
    tmp["stock"] = tkr
    tmp["news_titles"] = []
    
    news_infos = stk_infos.tickers[tkr].news
    
    for news in news_infos:
        tmp['news_titles'].append(news["title"])
        
    text[f"{CONFIG.etf_tkr} news"].append(tmp)
    
text = str(text)

print(text)

/home/moong/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'AAPL'

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:02<00:00,  1.93it/s]

{'AIQ news': [{'stock': 'AAPL', 'news_titles': ['Dow Jones Futures Fall; S&P 500 Struggles At Key Level As Apple Unveils iPhone 15', 'Apple cuts prices for iPhone 14 series in China -website', 'iPhone 15 and 15 Pro First Look: Why a Tiny USB-C Port Is a Huge Deal', 'Apple’s iPhone Price Bump Is Part of Subtle Revenue-Boosting Strategy', 'Tim Cook Surprises Apple Fans With His Acting Debut at the iPhone 15 Reveal', 'Apple Watch Series 9 and Apple Watch Ultra 2 hands-on', 'Get ready to replace all your iPhone charging cables because Apple just officially killed the Lightning connector', 'Apple’s iPhone 15 Pro Max Pushes Up the Price']}, {'stock': 'MSFT', 'news_titles': ['CPI data, AI tech forum: What to watch on Wednesday', "Google's search engine deals could be 'the heart' of antitrust case: Expert", 'Microsoft Powers Up with Helion Fusion: A Game-Changer in Clean Energy', 'Top 15 Sports Tech Companies And Startups In The World', 'Hard-Core Gamers Rejoice: New Xbox Mastercard Is Arrivin

### 03 프롬프트 엔지니어링

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI


template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                f"""
                Summarize the common issues based on the news headlines of stocks which are best 5 influential components of {CONFIG.etf_tkr} in Korean.
                
                - Tone : Humble
                - Style : MECE, accurate
                - Reader level : college student
                - Length : within 500 characters
                - Perspective : Mckinsey&Company senior consultant
                - Format : markdown
                
                You MUST answer in Korean. 
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

answer = llm(template.format_messages(text=text)).content

print(answer)

AIQ의 주요 구성요소인 AAPL, MSFT, NVDA, GOOGL, AMZN 주식에 대한 뉴스 헤드라인을 기반으로 일반적인 문제를 요약하면 다음과 같습니다.

AAPL:
- iPhone 15 시리즈 출시로 인한 가격인하는 중국에서도 이뤄지고 있음
- iPhone 15 Pro Max의 가격 상승

MSFT:
- AI 기술 포럼과 CPI 데이터에 대한 관심
- 구글의 검색 엔진 계약이 과징금 사례의 핵심일 수 있음
- Helion Fusion을 통한 깨끗한 에너지 발전 기술 도입

NVDA:
- 주식 추천 및 주식 매매에 대한 정보 제공
- NVIDIA의 CEO가 주식을 판매함
- Musk와 Zuckerberg의 AI에 대한 대화가 주목받고 있음

GOOGL:
- 구글의 검색 독점을 입증하기 위해 법무부가 노력 중
- 구글의 검색 엔진 계약이 과징금 사례의 핵심일 수 있음
- AI 기술이 애플의 iPhone과 워치에 조용히 적용되고 있음

AMZN:
- 계약된 배송 파트너에게 더 높은 급여 제공 계획
- 판매자를 위한 엔드 투 엔드 공급망 서비스 확장
- 대형 기업들을 이기는 작은 기업들의 등장

이러한 뉴스 헤드라인을 종합해보면, 각 주식에 대한 가격 변동, 기술 도입, 경쟁사와의 관계 등이 주요 문제로 부각되고 있음을 알 수 있습니다.
